## 测试基于 clickhouse 的策略管理系统接口

---

In [ ]:
import sys
sys.path.insert(0, "A:/ZB/git_repo/waditu/czsc")
import dotenv
dotenv.load_dotenv(dotenv_path="A:/ZB/git_repo/waditu/czsc/.env", override=True)

import os
import czsc
import pandas as pd

czsc.welcome()

database = "czsc_strategy_test"
## 初始化策略管理数据库

czsc.cwc.initialize(database=database)

C:\Users\zengb\.czsc 目录缓存超过1GB，请适当清理。调用 czsc.empty_cache_path() 可以直接清空缓存
欢迎使用CZSC！当前版本标识为 0.10.3@20251003

世界不是用来解释的，世界是用来改变的，“患”以“不患”的无位次而位次，而不同位次的实践，就是
人对世界的改变，从而才有“人不知”的世界到“人不愠”的世界。

--摘自《2006-11-19 12:12 《论语》详解：给所有曲解孔子的人（32）》
CZSC环境变量：czsc_min_bi_len = 6; czsc_max_bi_num = 50; 
C:\Users\zengb\.czsc 目录缓存超过1GB，请适当清理。调用 czsc.empty_cache_path() 可以直接清空缓存


2025-10-03 12:09:48.387 | INFO     | czsc.traders.cwc:init_tables:293 - metas 表创建成功！
2025-10-03 12:09:48.437 | INFO     | czsc.traders.cwc:init_tables:295 - weights 表创建成功！
2025-10-03 12:09:48.476 | INFO     | czsc.traders.cwc:init_tables:297 - returns 表创建成功！
2025-10-03 12:09:48.521 | INFO     | czsc.traders.cwc:init_latest_weights_view:193 - cs_latest_weights 视图初始化完成
2025-10-03 12:09:48.569 | INFO     | czsc.traders.cwc:init_latest_weights_view:220 - ts_latest_weights 视图初始化完成
2025-10-03 12:09:48.613 | INFO     | czsc.traders.cwc:init_latest_weights_view:230 - latest_weights 视图初始化完成
2025-10-03 12:09:48.614 | INFO     | czsc.traders.cwc:init_latest_weights_view:232 - 所有最新持仓权重视图初始化完成


### 策略数据写入

---

1. 写入策略元数据
2. 写入策略持仓权重
3. 写入策略日收益


In [2]:
czsc.cwc.set_meta(
    strategy="测试策略1号",
    base_freq="日线",
    description="这是一个测试用的策略",
    author="xxx",
    outsample_sdt="20220101",
    weight_type="ts",
    status="实盘",
    memo="",
    overwrite=False,
    database=database,
)

2025-10-03 12:09:54.398 | WARNING  | czsc.traders.cwc:set_meta:388 - 策略 测试策略1号 已存在元数据，如需更新请设置 overwrite=True


In [3]:
dfw = czsc.mock.generate_klines_with_weights()

czsc.cwc.publish_weights(df=dfw, strategy="测试策略1号", database=database)

2025-10-03 12:09:55.156 | INFO     | czsc.utils.cache:is_found:74 - 缓存文件已找到, C:\Users\zengb\.czsc\generate_klines\387FD5BE_generate_klines.pkl
2025-10-03 12:09:55.157 | INFO     | czsc.utils.cache:get:85 - 正在读取缓存记录，地址：C:\Users\zengb\.czsc\generate_klines\387FD5BE_generate_klines.pkl
2025-10-03 12:09:56.114 | INFO     | czsc.traders.cwc:__send_heartbeat:435 - 策略 测试策略1号 发送心跳成功
2025-10-03 12:09:56.222 | INFO     | czsc.traders.cwc:publish_weights:533 - 策略 测试策略1号 最新时间：2025-06-08 00:00:00+08:00
2025-10-03 12:09:56.239 | INFO     | czsc.traders.cwc:publish_weights:544 - 策略 测试策略1号 共 0 条新信号
2025-10-03 12:09:56.241 | INFO     | czsc.traders.cwc:publish_weights:552 - 准备发布 0 条策略信号
2025-10-03 12:09:56.241 | INFO     | czsc.traders.cwc:publish_weights:566 - 完成所有信号发布, 共 0 条
2025-10-03 12:09:56.304 | WARNING  | czsc.traders.cwc:get_meta:329 - 策略 测试策略1号 不存在元数据
2025-10-03 12:09:56.305 | WARNING  | czsc.traders.cwc:__send_heartbeat:428 - 策略 测试策略1号 不存在元数据，无法发送心跳


In [18]:
wb = czsc.WeightBacktest(dfw, digits=2, fee_rate=0.0)
dfr = pd.melt(wb.daily_return.copy(), id_vars=["date"], var_name="symbol", value_name="returns")
dfr.rename(columns={"date": "dt"}, inplace=True)

print(dfr.tail())

czsc.cwc.publish_returns(df=dfr, strategy="测试策略1号", database=database)

                dt symbol   returns
101479  2025-06-04  total -0.000938
101480  2025-06-05  total  0.006439
101481  2025-06-06  total  0.000481
101482  2025-06-07  total -0.000889
101483  2025-06-08  total  0.000000


2025-10-03 12:18:19.245 | INFO     | czsc.traders.cwc:publish_returns:601 - 策略 测试策略1号 最新时间：2025-06-08 00:00:00+08:00
2025-10-03 12:18:19.264 | INFO     | czsc.traders.cwc:publish_returns:612 - 策略 测试策略1号 共 18 条新日收益
2025-10-03 12:18:19.268 | INFO     | czsc.traders.cwc:publish_returns:620 - 准备发布 18 条策略日收益
2025-10-03 12:18:19.361 | INFO     | czsc.traders.cwc:publish_returns:628 - 完成批次 1, 发布 18 条日收益
2025-10-03 12:18:19.363 | INFO     | czsc.traders.cwc:publish_returns:633 - 完成所有日收益发布, 共 18 条


### 读取策略数据
---

In [21]:
# 读取策略列表

dfm = czsc.cwc.get_all_metas(database=database)
dfm

,strategy,base_freq,description,author,outsample_sdt,create_time,update_time,heartbeat_time,weight_type,status,memo
0,测试策略1号,日线,这是一个测试用的策略,xxx,2022-01-01 00:00:00+08:00,2025-10-03 11:44:10+08:00,2025-10-03 11:44:10+08:00,2025-10-03 12:09:55+08:00,ts,实盘,


In [26]:
dfw = czsc.cwc.get_strategy_weights(strategy="测试策略1号", database=database)
dfw

,dt,symbol,weight,strategy,update_time
0,2010-01-01 00:00:00+08:00,000001,-1.000000,测试策略1号,2025-10-03 11:44:47+08:00
1,2010-01-01 00:00:00+08:00,AAPL,-0.811411,测试策略1号,2025-10-03 11:44:47+08:00
2,2010-01-01 00:00:00+08:00,AMZN,-1.000000,测试策略1号,2025-10-03 11:44:47+08:00
3,2010-01-01 00:00:00+08:00,BTC,-0.534633,测试策略1号,2025-10-03 11:44:47+08:00
4,2010-01-01 00:00:00+08:00,CSCO,0.841115,测试策略1号,2025-10-03 11:44:47+08:00
...,...,...,...,...,...
95841,2025-06-08 00:00:00+08:00,NVDA,-0.842777,测试策略1号,2025-10-03 11:44:47+08:00
95842,2025-06-08 00:00:00+08:00,ORCL,-1.000000,测试策略1号,2025-10-03 11:44:47+08:00
95843,2025-06-08 00:00:00+08:00,PYPL,0.256551,测试策略1号,2025-10-03 11:44:47+08:00
95844,2025-06-08 00:00:00+08:00,SAP,-0.052823,测试策略1号,2025-10-03 11:44:47+08:00


In [27]:
dfw = czsc.cwc.get_strategy_weights(strategy="测试策略1号", database=database, sdt="20230101")
dfw

,dt,symbol,weight,strategy,update_time
0,2023-01-01 00:00:00+08:00,000001,-1.000000,测试策略1号,2025-10-03 11:44:47+08:00
1,2023-01-01 00:00:00+08:00,AAPL,-1.000000,测试策略1号,2025-10-03 11:44:47+08:00
2,2023-01-01 00:00:00+08:00,AMZN,-0.268448,测试策略1号,2025-10-03 11:44:47+08:00
3,2023-01-01 00:00:00+08:00,BTC,-1.000000,测试策略1号,2025-10-03 11:44:47+08:00
4,2023-01-01 00:00:00+08:00,CSCO,-1.000000,测试策略1号,2025-10-03 11:44:47+08:00
...,...,...,...,...,...
15125,2025-06-08 00:00:00+08:00,NVDA,-0.842777,测试策略1号,2025-10-03 11:44:47+08:00
15126,2025-06-08 00:00:00+08:00,ORCL,-1.000000,测试策略1号,2025-10-03 11:44:47+08:00
15127,2025-06-08 00:00:00+08:00,PYPL,0.256551,测试策略1号,2025-10-03 11:44:47+08:00
15128,2025-06-08 00:00:00+08:00,SAP,-0.052823,测试策略1号,2025-10-03 11:44:47+08:00


In [ ]:

dfr = czsc.cwc.get_strategy_returns(strategy="测试策略1号", database=database)
dfr

,dt,symbol,returns,strategy,update_time
0,2010-01-01 00:00:00+08:00,000001,0.0,测试策略1号,2025-10-03 12:16:07+08:00
1,2010-01-01 00:00:00+08:00,AAPL,0.0,测试策略1号,2025-10-03 12:16:07+08:00
2,2010-01-01 00:00:00+08:00,AMZN,0.0,测试策略1号,2025-10-03 12:16:07+08:00
3,2010-01-01 00:00:00+08:00,BTC,0.0,测试策略1号,2025-10-03 12:16:07+08:00
4,2010-01-01 00:00:00+08:00,CSCO,0.0,测试策略1号,2025-10-03 12:16:07+08:00
...,...,...,...,...,...
101479,2025-06-08 00:00:00+08:00,ORCL,0.0,测试策略1号,2025-10-03 12:18:19+08:00
101480,2025-06-08 00:00:00+08:00,PYPL,0.0,测试策略1号,2025-10-03 12:18:19+08:00
101481,2025-06-08 00:00:00+08:00,SAP,0.0,测试策略1号,2025-10-03 12:18:19+08:00
101482,2025-06-08 00:00:00+08:00,TSLA,0.0,测试策略1号,2025-10-03 12:18:19+08:00


In [33]:
dfl  = czsc.cwc.get_latest_weights(database=database)
dfl

,dt,symbol,weight,strategy,update_time
0,2025-06-08 00:00:00+08:00,000001,-0.149599,测试策略1号,2025-10-03 11:44:47+08:00
1,2025-06-08 00:00:00+08:00,AAPL,-1.000000,测试策略1号,2025-10-03 11:44:47+08:00
2,2025-06-08 00:00:00+08:00,AMZN,-1.000000,测试策略1号,2025-10-03 11:44:47+08:00
3,2025-06-08 00:00:00+08:00,BTC,-1.000000,测试策略1号,2025-10-03 11:44:47+08:00
4,2025-06-08 00:00:00+08:00,CSCO,-0.285907,测试策略1号,2025-10-03 11:44:47+08:00
5,2025-06-08 00:00:00+08:00,ETH,-1.000000,测试策略1号,2025-10-03 11:44:47+08:00
6,2025-06-08 00:00:00+08:00,GOOGL,-0.811283,测试策略1号,2025-10-03 11:44:47+08:00
7,2025-06-08 00:00:00+08:00,IBM,-0.910074,测试策略1号,2025-10-03 11:44:47+08:00
8,2025-06-08 00:00:00+08:00,INTC,-1.000000,测试策略1号,2025-10-03 11:44:47+08:00
9,2025-06-08 00:00:00+08:00,META,-1.000000,测试策略1号,2025-10-03 11:44:47+08:00
